# 使用Char-RNN生成莎士比亚文本
## 创建训练数据集

In [1]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

首先我们使用keras的get_file()函数来下载莎士比亚的所有作品，并从Andrej Karpathy的Char-RNN项目下载数据。

In [2]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_txt=f.read()

首先我们需要为文本添加一个分词器：它会找到文本中使用的所有字符，并将其映射到不同的字符ID。从1开始到不同字符数量。（它不从0开始，所以我们可以用该值进行屏蔽）。

In [3]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_txt])

我们设置char_level=True来得到字符级编码，而不是默认的单词级编码。请注意默认情况下，该分词器将文本转换为小写(不希望这样做可以设置lower=False)。现在分词器可以将一个句子(或句子列表)编码为字符ID列表并返回。

In [4]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [5]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [6]:
max_id=len(tokenizer.word_index) # 不同字符的数量
max_id

39

In [7]:
dataset_size=tokenizer.document_count
dataset_size

1

让我们对全文进行编码，以便每个字符都由其ID进行表示(我们减去1即可得到从0到38的ID)。

In [8]:
[encoded]=np.array(tokenizer.texts_to_sequences([shakespeare_txt]))-1

In [9]:
encoded[:10]

array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5])

## 如何拆分顺序数据集
将时间序列划分成训练集、验证集和测试集很大程度上取决于你的任务。

In [10]:
train_size=encoded.shape[0]*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
encoded.shape[0],train_size

(1115394, 1003854)

## 将顺序数据集切成多个窗口
训练集现在由超过100万个字符的单词序列组成，所以我们不能直接在其上训练神经网络：RNN相当于一个超过100万层的深层网络。取而代之的是，我们会使用数据集的window()方法将这个长字符序列转换为许多较小的文本窗口。数据集的每个实例将是整个文本的很短的子字符串，并且RNN仅在这些子字符串的长度上展开。这称为时间截断反向传播。

In [12]:
n_steps=100
window_length=n_steps+1 # 目标 = 输入向前移动 1 个字符
dataset=dataset.window(window_length,shift=1,drop_remainder=True)

In [13]:
dataset

<WindowDataset shapes: DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([])), types: DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([]))>

较短的输入序列上训练RNN会更容易，但是当然RNN不能学习比n_steps长的任何模式，因此不要使其太短。

默认情况下，window会创建不重叠的窗口，但是为了获得最大可能的训练集，我们使用shift=1，以便使第一个窗口包含0到100的字符，第二个窗口包含1到101的字符，以此类推。为确保所有窗口正好是101个字符长度，我们设置drop_remainder=True。